In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [53]:
BASE_URL = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok='

In [54]:
r = requests.get(BASE_URL+str(1)+'&page='+str(1))

In [55]:
soup = BeautifulSoup(r.text, 'lxml')

In [56]:
total_page_num = soup.select_one('td.pgRR > a')
total_page_num  = int(total_page_num.get('href').split('=')[-1])
total_page_num

30

In [57]:
ipt_html = soup.select_one('div.subcnt_sise_item_top')
global fields
fields = [item.get('value') for item in ipt_html.select('input') if item.get('value') in ['per', 'pbr', 'roe', 'market_sum', 'property_total', 'debt_total', 'sales', 'reserve_ratio']]
fields

['market_sum',
 'per',
 'property_total',
 'roe',
 'debt_total',
 'sales',
 'pbr',
 'reserve_ratio']

In [58]:
global fields
data = {
    'menu': 'market_sum',
    'returnUrl': BASE_URL + str(1) + "&page=" + str(1),
    'fieldIds': fields
}

r2 = requests.post('https://finance.naver.com/sise/field_submit.nhn', data=data)
r2.encoding='euc-kr'
r2.text

bl2.gif" width="15" height="13" alt="토론실"></a></td>\n\t\t\t\t</tr>\n\t\t\t\n\t\n\t\t\n\t\t\n\t\n\n\n\t\t\t\t<tr  onMouseOver="mouseOver(this)" onMouseOut="mouseOut(this)">\n\t\t\t\t\t<td class="no">43</td>\n\t\t\t\t\t<td><a href="/item/main.nhn?code=035900" class="tltle">JYP Ent.</a></td>\n\t\t\t\t\t<td class="number">34,900</td>\n\t\t\t\t\t<td class="number">\n\t\t\t\t<img src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" width="7" height="6" style="margin-right:4px;" alt="상승"><span class="tah p11 red02">\n\t\t\t\t200\n\t\t\t\t</span>\n\t\t\t</td>\n\t\t\t\t\t<td class="number">\n\t\t\t\t<span class="tah p11 red01">\n\t\t\t\t+0.58%\n\t\t\t\t</span>\n\t\t\t</td>\n\t\t\t\t\t<td class="number">500</td>\n\t\n\t\t\n\t\t\t\n\t\t\t\n\t\t\t\n\t\t\t\t\t\t\t\t\t<td class="number">12,389</td>\n\t\t\t\n\t\t\n\t\n\t\t\n\t\t\t\n\t\t\t\n\t\t\t\n\t\t\t\t\t\t\t\t\t<td class="number">1,444</td>\n\t\t\t\n\t\t\n\t\n\t\t\n\t\t\t\n\t\t\t\n\t\t\t\n\t\t\t\t\t\t\t\t\t<td class="number">2,239</td

In [110]:
soup2 = BeautifulSoup(r2.text, 'lxml')
# table
table_html = soup2.select_one('table.type_2')
soup2
with open('test.txt', 'w') as f:
    f.write(str(soup2))

In [261]:
# column
header_data = [item.get_text().strip() for item in table_html.find('thead').find('tr').find_all('th') if item.get_text().strip() not in ['전일비', '등락률']]
header_data

['N',
 '종목명',
 '현재가',
 '액면가',
 '시가총액',
 '매출액',
 '자산총계',
 '부채총계',
 'PER',
 'ROE',
 'PBR',
 '유보율',
 '토론실']

In [262]:
del header_data[-1]

In [263]:
# 첫 줄은 필요없는 줄
td_items = [tr_item.find_all('td') for tr_item in table_html.find('tbody').find_all('tr')][1:]

In [342]:
inner_data = []
for item in td_items:
    for i in item[0:len(item)-1]:
        if i.find('a') or len(i.get('class', [])) > 0:
            child = i.findChildren()
            if len(child) >= 1:
                if child[0].get('href'):
                    inner_data.append(child[0].get_text().strip())
            else:
                inner_data.append(i.get_text().strip().replace(',', ''))

In [252]:
no_data = [i.get_text().strip() for item in td_items for i in item if len(i.get('class', [])) > 0 and ('no' in i.get('class', []))] # 번호

In [253]:
number_data = np.array(inner_data)

In [254]:
number_data.resize(len(no_data), len(header_data))

In [255]:
df = pd.DataFrame(data=number_data, columns=header_data)

In [2]:
pd_result = pd.read_excel('result.xlsx')

In [3]:
pd_result = pd_result.dropna()

In [4]:
pd_result.describe()

,N,현재가,액면가,상장주식수,시가총액,매출액,자산총계,부채총계,PER,ROE,PBR,유보율
count,775.000000,7.750000e+02,775.000000,7.750000e+02,7.750000e+02,7.750000e+02,7.750000e+02,7.750000e+02,775.000000,775.000000,775.000000,775.000000
mean,480.409032,4.750273e+04,2089.290323,6.994901e+04,2.591846e+04,3.227523e+04,9.432642e+04,7.076774e+04,43.187342,-2.941329,1.715252,3167.330581
std,296.477658,1.089494e+05,2074.784354,2.390911e+05,1.932658e+05,1.198899e+05,4.713243e+05,4.092072e+05,823.692149,53.104103,4.332177,8739.029755
min,1.000000,2.580000e+02,0.000000,4.080000e+02,1.940000e+02,2.200000e+01,2.650000e+02,0.000000e+00,-6170.000000,-1165.400000,-4.710000,-1857.500000
25%,225.500000,4.430000e+03,500.000000,1.323850e+04,1.280500e+03,1.714000e+03,2.529000e+03,8.730000e+02,-2.565000,-1.780000,0.580000,282.100000
50%,469.000000,1.220000e+04,500.000000,2.870500e+04,2.947000e+03,4.786000e+03,6.495000e+03,2.776000e+03,10.150000,3.840000,0.940000,1052.000000
75%,723.500000,4.292500e+04,5000.000000,6.299350e+04,9.454000e+03,1.659050e+04,2.355250e+04,1.180800e+04,22.800000,8.935000,1.865000,2684.800000
max,1177.000000,1.570000e+06,5000.000000,5.969783e+06,5.062376e+06,2.368070e+06,6.106722e+06,5.673107e+06,21361.100000,61.960000,107.420000,127125.600000


In [5]:
pd_result['BPS'] = pd_result['현재가'] / pd_result['PBR']

In [6]:
pd_result['부채비율'] = (pd_result['부채총계']/pd_result['자산총계']) * 100

In [7]:
pd_result['부채비율'].head()

0    27.043375
1    27.067225
2    51.481116
4    54.600388
5    63.533262
Name: 부채비율, dtype: float64

In [8]:
pd_result['BPS'].head()

0     39441.860465
1     75806.451613
2     49934.210526
4    235227.272727
5    268390.804598
Name: BPS, dtype: float64

In [9]:
pd_result['ROE']

0        9.99
1        9.53
2       15.22
4        2.93
5        2.04
        ...  
1120    -7.13
1137   -30.21
1158    -4.28
1172   -43.44
1176   -15.33
Name: ROE, Length: 775, dtype: float64

In [10]:
pd_result['PER x PBR'] = pd_result['PER'] * pd_result['PBR']

In [11]:
pd_result['PER x PBR'].head()

0     47.4720
1     40.1574
2    473.0240
4    445.0336
5     39.4980
Name: PER x PBR, dtype: float64

In [12]:
pd_result['1/PER'] = 1/pd_result['PER'].astype(float)

In [13]:
pd_result['BPR'] = 1/pd_result['PBR'].astype(float)

In [15]:
pd_result['RANK_BPR'] = pd_result['BPR'].rank(method='max', ascending=False)

In [16]:
pd_result['RANK_1/PER'] = pd_result['1/PER'].rank(method='max', ascending=False)

In [19]:
pd_result['RANK_VALUE'] = (pd_result['RANK_BPR'] + pd_result['RANK_1/PER']) / 2

In [20]:
pd_result = pd_result.sort_values(by=['RANK_VALUE'])

In [24]:
pd_result['종목명'][:20]

130    BNK금융지주
865       세원정공
645      한솔홀딩스
501     DB금융투자
467         E1
498     KPX홀딩스
170    DGB금융지주
49        기업은행
348       서울가스
181     JB금융지주
515        아세아
883       삼정펄프
139        KCC
822     경동인베스트
537       한신공영
46      우리금융지주
535         대덕
545       대성산업
279        HDC
25        한국전력
Name: 종목명, dtype: object

In [21]:
pd_result.loc[:75]

,N,종목명,현재가,액면가,상장주식수,시가총액,매출액,자산총계,부채총계,PER,...,PBR,유보율,BPS,부채비율,PER x PBR,1/PER,BPR,RANK_BPR,RANK_1/PER,RANK_VALUE
130,131,BNK금융지주,6570,5000,325935,21414,52124.0,1141569.0,1045306.0,4.12,...,0.24,491.1,27375.000000,91.567483,0.9888,0.242718,4.166667,12.0,27.0,19.5
865,866,세원정공,8090,500,10000,809,1569.0,6128.0,948.0,4.90,...,0.19,8152.6,42578.947368,15.469974,0.9310,0.204082,5.263158,4.0,45.0,24.5
645,646,한솔홀딩스,3835,1000,42009,1611,3595.0,6271.0,1007.0,2.36,...,0.32,1076.9,11984.375000,16.058045,0.7552,0.423729,3.125000,42.0,8.0,25.0
501,502,DB금융투자,6110,5000,42446,2593,15903.0,99596.0,90375.0,2.69,...,0.32,338.0,19093.750000,90.741596,0.8608,0.371747,3.125000,42.0,10.0,26.0
467,468,E1,43150,5000,6860,2960,39276.0,37314.0,24880.0,4.91,...,0.21,3562.2,205476.190476,66.677387,1.0311,0.203666,4.761905,7.0,46.0,26.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,463,이아이디,387,200,773343,2993,1652.0,2259.0,786.0,-10.18,...,1.95,9.9,198.461538,34.794157,-19.8510,-0.098232,0.512821,591.0,679.0,635.0
369,370,덕성,25950,500,15680,4069,1177.0,1050.0,388.0,122.41,...,6.07,733.5,4275.123558,36.952381,743.0287,0.008169,0.164745,746.0,525.0,635.5
646,647,한국화장품제조,35250,500,4532,1598,620.0,777.0,334.0,-39.74,...,2.89,1891.3,12197.231834,42.985843,-114.8486,-0.025164,0.346021,675.0,596.0,635.5
77,78,한미약품,317000,2500,12078,38288,10759.0,18820.0,10501.0,320.20,...,5.21,2526.0,60844.529750,55.797024,1668.2420,0.003123,0.191939,735.0,539.0,637.0


In [430]:
condition_result = pd_result[(pd_result['PER'] <= 10.0) & (pd_result['PBR'] < 1.0) & (pd_result['ROE'] > 5.0) & (pd_result['시가총액'] >= 10000.0 ) & (pd_result['부채비율'] <= 100.0) & (pd_result['BPS'] > pd_result['현재가'])]

In [431]:
condition_result

,N,종목명,현재가,액면가,상장주식수,시가총액,매출액,자산총계,부채총계,PER,ROE,PBR,유보율,BPS,부채비율,PER x PBR
17,18,KB금융,53900,5000,415808,224120,556802.0,6106722.0,5673107.0,6.49,8.53,0.49,2028.5,110000.000000,92.899382,3.1801
21,22,신한지주,36350,5000,516600,187784,290007.0,6052341.0,5588772.0,5.39,8.20,0.44,1483.4,82613.636364,92.340666,2.3716
28,29,하나금융지주,41150,5000,300242,123550,482159.0,4603133.0,4287130.0,4.68,8.96,0.39,2025.0,105512.820513,93.135045,1.8252
46,47,우리금융지주,10000,5000,722268,72227,286376.0,3990810.0,3723552.0,5.52,5.87,0.31,640.0,32258.064516,93.303164,1.7112
49,50,기업은행,9040,5000,738664,66775,185746.0,3616162.0,3364738.0,4.51,6.44,0.27,501.0,33481.481481,93.047214,1.2177
52,53,미래에셋대우,9710,5000,635316,61689,168358.0,1309469.0,1216007.0,9.51,8.94,0.68,136.8,14279.411765,92.862603,6.4668
59,60,한국금융지주,89900,5000,55726,50098,165117.0,704377.0,648054.0,6.41,16.27,0.94,1745.4,95638.297872,92.003856,6.0254
74,75,삼성카드,33900,5000,115859,39276,32934.0,220382.0,151418.0,9.53,5.04,0.51,1087.4,66470.588235,68.707063,4.8603
85,86,삼성증권,39350,5000,89300,35140,66562.0,447296.0,397803.0,7.47,8.15,0.68,979.5,57867.647059,88.935068,5.0796
91,92,DB손해보험,46500,500,70800,32922,186761.0,557778.0,496287.0,5.92,6.92,0.42,17354.1,110714.285714,88.975722,2.4864


In [432]:
condition_result['종목명']

17          KB금융
21          신한지주
28        하나금융지주
46        우리금융지주
49          기업은행
52        미래에셋대우
59        한국금융지주
74          삼성카드
85          삼성증권
91        DB손해보험
93        NH투자증권
97         메리츠증권
111         대우건설
127        메리츠화재
130      BNK금융지주
132         현대해상
133      메리츠금융지주
139          KCC
143    HDC현대산업개발
154       한국앤컴퍼니
156        코오롱인더
163       롯데정밀화학
170      DGB금융지주
181       JB금융지주
189         LG상사
190         다우기술
193      SK디스커버리
211         코리안리
Name: 종목명, dtype: object